In [1]:
import folium
from folium import plugins
import pandas as pd
import numpy as np
import requests
import json
import sys
sys.path.append('../src')

from data_collection import FinancialDataCollector
from data_processing import FinancialDataProcessor

In [2]:
# Load data
collector = FinancialDataCollector()
stock_data = collector.load_stock_data()

Data loaded from data\raw\stock_data.pkl


In [3]:
# Company headquarters data (manual for this example)
company_locations = {
    'AAPL': {'name': 'Apple Inc.', 'lat': 37.3348, 'lon': -122.0090, 'city': 'Cupertino, CA'},
    'MSFT': {'name': 'Microsoft', 'lat': 47.6444, 'lon': -122.1282, 'city': 'Redmond, WA'},
    'GOOGL': {'name': 'Alphabet', 'lat': 37.4239, 'lon': -122.0827, 'city': 'Mountain View, CA'},
    'AMZN': {'name': 'Amazon', 'lat': 47.6205, 'lon': -122.3493, 'city': 'Seattle, WA'},
    'TSLA': {'name': 'Tesla', 'lat': 30.2240, 'lon': -97.6147, 'city': 'Austin, TX'},
    'META': {'name': 'Meta', 'lat': 37.4850, 'lon': -122.1477, 'city': 'Menlo Park, CA'},
    'NVDA': {'name': 'NVIDIA', 'lat': 37.3708, 'lon': -121.9781, 'city': 'Santa Clara, CA'},
    'JPM': {'name': 'JPMorgan Chase', 'lat': 40.7614, 'lon': -73.9776, 'city': 'New York, NY'},
    'JNJ': {'name': 'Johnson & Johnson', 'lat': 40.4958, 'lon': -74.4433, 'city': 'New Brunswick, NJ'},
    'V': {'name': 'Visa', 'lat': 37.4419, 'lon': -122.1430, 'city': 'Foster City, CA'},
    'WMT': {'name': 'Walmart', 'lat': 36.3719, 'lon': -94.2088, 'city': 'Bentonville, AR'},
    'PG': {'name': 'Procter & Gamble', 'lat': 39.1612, 'lon': -84.4569, 'city': 'Cincinnati, OH'},
    'HD': {'name': 'Home Depot', 'lat': 33.7490, 'lon': -84.3880, 'city': 'Atlanta, GA'},
    'MA': {'name': 'Mastercard', 'lat': 40.9523, 'lon': -73.7772, 'city': 'Purchase, NY'},
    'BAC': {'name': 'Bank of America', 'lat': 35.2271, 'lon': -80.8431, 'city': 'Charlotte, NC'}
}


In [5]:
# Plot-1. Market Cap Bubble Map
def create_market_cap_map():
    # Create base map centered on US
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles='OpenStreetMap')
    
    # Add company markers with market cap sizing
    for ticker, location in company_locations.items():
        if ticker in stock_data:
            market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9  # In billions
            
            # Calculate recent performance
            prices = stock_data[ticker]['prices']['Close']
            recent_change = ((prices.iloc[-1] - prices.iloc[-30]) / prices.iloc[-30]) * 100
            
            # Color based on performance
            color = 'green' if recent_change > 0 else 'red'
            
            # Size based on market cap
            radius = max(10, min(50, market_cap / 20))
            
            # Create popup text
            popup_text = f"""
            <b>{location['name']} ({ticker})</b><br>
            Location: {location['city']}<br>
            Market Cap: ${market_cap:.1f}B<br>
            30-Day Change: {recent_change:.1f}%
            """
            
            folium.CircleMarker(
                location=[location['lat'], location['lon']],
                radius=radius,
                popup=folium.Popup(popup_text, max_width=300),
                color='black',
                fillColor=color,
                fillOpacity=0.7,
                weight=2
            ).add_to(m)
    
    # Add legend
    legend_html = """
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <p><b>Market Cap & Performance</b></p>
    <p><i class="fa fa-circle" style="color:green"></i> Positive 30-day return</p>
    <p><i class="fa fa-circle" style="color:red"></i> Negative 30-day return</p>
    <p>Bubble size = Market Cap</p>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    
    m.save('./static/folium_market_cap_map.html')
    return m

market_cap_map = create_market_cap_map()

In [6]:
# Plot-2. Performance Heat Map
def create_performance_heatmap():
    # Prepare data for heat map
    heat_data = []
    
    for ticker, location in company_locations.items():
        if ticker in stock_data:
            prices = stock_data[ticker]['prices']['Close']
            ytd_change = ((prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]) * 100
            
            # Create heat point (lat, lon, weight)
            weight = max(0.1, min(1.0, (ytd_change + 50) / 100))  # Normalize to 0-1
            heat_data.append([location['lat'], location['lon'], weight])
    
    # Create map
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    
    # Add heat map
    plugins.HeatMap(heat_data, radius=50, blur=25, gradient={
        0.0: 'blue', 0.3: 'lime', 0.5: 'yellow', 0.7: 'orange', 1.0: 'red'
    }).add_to(m)
    
    # Add markers for reference
    for ticker, location in company_locations.items():
        if ticker in stock_data:
            prices = stock_data[ticker]['prices']['Close']
            ytd_change = ((prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]) * 100
            
            folium.Marker(
                location=[location['lat'], location['lon']],
                popup=f"{location['name']}<br>YTD: {ytd_change:.1f}%",
                icon=folium.Icon(color='white', icon='info-sign')
            ).add_to(m)
    
    m.save('./static/folium_performance_heatmap.html')
    return m

performance_heatmap = create_performance_heatmap()

In [7]:
# Plot-3. Sector Clustering Map
def create_sector_clustering_map():
    from config import SECTOR_MAPPING
    
    # Create map
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    
    # Color mapping for sectors
    sector_colors = {
        'Technology': 'blue',
        'Healthcare': 'green', 
        'Financial': 'red',
        'Consumer': 'purple',
        'Energy': 'orange',
        'Communication': 'pink',
        'Industrial': 'gray',
        'Materials': 'brown',
        'Utilities': 'lightgreen',
        'Consumer Staples': 'darkblue'
    }
    
    # Add markers by sector
    for sector, tickers in SECTOR_MAPPING.items():
        sector_group = folium.FeatureGroup(name=sector)
        
        for ticker in tickers:
            if ticker in company_locations and ticker in stock_data:
                location = company_locations[ticker]
                market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
                
                popup_text = f"""
                <b>{location['name']}</b><br>
                Sector: {sector}<br>
                Market Cap: ${market_cap:.1f}B
                """
                
                folium.Marker(
                    location=[location['lat'], location['lon']],
                    popup=folium.Popup(popup_text, max_width=250),
                    icon=folium.Icon(color=sector_colors.get(sector, 'gray'))
                ).add_to(sector_group)
        
        sector_group.add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    m.save('./static/folium_sector_clustering.html')
    return m

sector_clustering_map = create_sector_clustering_map()

In [8]:
# Plot-4. Interactive Choropleth (State-level analysis)
def create_state_choropleth():
    # State-level aggregation
    state_data = {
        'CA': {'companies': 6, 'total_market_cap': 0, 'avg_performance': 0},
        'WA': {'companies': 2, 'total_market_cap': 0, 'avg_performance': 0},
        'TX': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0},
        'NY': {'companies': 2, 'total_market_cap': 0, 'avg_performance': 0},
        'NJ': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0},
        'AR': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0},
        'OH': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0},
        'GA': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0},
        'NC': {'companies': 1, 'total_market_cap': 0, 'avg_performance': 0}
    }
    
    # Aggregate data by state
    state_mapping = {
        'AAPL': 'CA', 'GOOGL': 'CA', 'META': 'CA', 'NVDA': 'CA', 'V': 'CA',
        'MSFT': 'WA', 'AMZN': 'WA', 'TSLA': 'TX', 'JPM': 'NY', 'MA': 'NY',
        'JNJ': 'NJ', 'WMT': 'AR', 'PG': 'OH', 'HD': 'GA', 'BAC': 'NC'
    }
    
    for ticker, state in state_mapping.items():
        if ticker in stock_data:
            market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
            prices = stock_data[ticker]['prices']['Close']
            ytd_performance = ((prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]) * 100
            
            state_data[state]['total_market_cap'] += market_cap
            state_data[state]['avg_performance'] += ytd_performance
    
    # Calculate averages
    for state in state_data:
        if state_data[state]['companies'] > 0:
            state_data[state]['avg_performance'] /= state_data[state]['companies']
    
    # Create DataFrame for choropleth
    choropleth_data = []
    for state, data in state_data.items():
        choropleth_data.append({
            'state': state,
            'companies': data['companies'],
            'total_market_cap': data['total_market_cap'],
            'avg_performance': data['avg_performance']
        })
    
    df_states = pd.DataFrame(choropleth_data)
    
    # Note: For a full choropleth, you'd need state boundary GeoJSON data
    # This creates a simple marker-based visualization instead
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    
    # State centers (approximate)
    state_centers = {
        'CA': [36.7783, -119.4179],
        'WA': [47.7511, -120.7401], 
        'TX': [31.9686, -99.9018],
        'NY': [42.1657, -74.9481],
        'NJ': [40.2206, -74.7564],
        'AR': [34.9513, -92.3809],
        'OH': [40.3888, -82.7649],
        'GA': [33.2490, -83.4426],
        'NC': [35.5175, -80.8164]
    }
    
    for state, data in state_data.items():
        if state in state_centers and data['companies'] > 0:
            # Color based on performance
            color = 'green' if data['avg_performance'] > 0 else 'red'
            
            # Size based on market cap
            radius = max(20, min(60, data['total_market_cap'] / 100))
            
            popup_text = f"""
            <b>{state}</b><br>
            Companies: {data['companies']}<br>
            Total Market Cap: ${data['total_market_cap']:.1f}B<br>
            Avg Performance: {data['avg_performance']:.1f}%
            """
            
            folium.CircleMarker(
                location=state_centers[state],
                radius=radius,
                popup=folium.Popup(popup_text, max_width=250),
                color='black',
                fillColor=color,
                fillOpacity=0.7,
                weight=3
            ).add_to(m)
    
    m.save('./static/folium_state_analysis.html')
    return m

state_choropleth = create_state_choropleth()

In [9]:
# Plot-5. Time-based Animation Map (Market Cap Evolution)
def create_time_animation_map():
    # Create time series data for animation
    animation_data = []
    
    # Get quarterly data points
    for ticker, location in list(company_locations.items())[:8]:  # Limit for performance
        if ticker in stock_data:
            prices = stock_data[ticker]['prices']['Close']
            market_cap_base = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
            
            # Sample every 30 days for animation frames
            for i in range(0, len(prices), 30):
                if i < len(prices):
                    date = prices.index[i]
                    price_ratio = prices.iloc[i] / prices.iloc[0]  # Relative to start
                    estimated_market_cap = market_cap_base * price_ratio
                    
                    animation_data.append({
                        'ticker': ticker,
                        'company': location['name'],
                        'lat': location['lat'],
                        'lon': location['lon'],
                        'date': date.strftime('%Y-%m-%d'),
                        'market_cap': estimated_market_cap,
                        'timestamp': date.timestamp()
                    })
    
    df_animation = pd.DataFrame(animation_data)
    
    # Create base map
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    
    # Add time series markers (showing latest data)
    latest_data = df_animation.groupby('ticker').last()
    
    for ticker, row in latest_data.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=max(10, min(40, row['market_cap'] / 50)),
            popup=f"{row['company']}<br>Est. Market Cap: ${row['market_cap']:.1f}B",
            color='blue',
            fillColor='lightblue',
            fillOpacity=0.7
        ).add_to(m)
    
    # Note: For full animation, you'd use folium.plugins.TimestampedGeoJson
    # This creates a static representation of the final state
    
    m.save('./static/folium_time_animation.html')
    return m

time_animation_map = create_time_animation_map()
